## Deliverable 2. Create a Customer Travel Destinations Map.

In [88]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key


# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [89]:
# 1. Import the WeatherPy_database.csv file. 

absolute_path = os.path.abspath(os.path.dirname('../Weather_Database/Weather_database.csv'))
print(absolute_path + 'Weather_database.csv')

city_data_df = pd.read_csv(absolute_path + '/Weather_database.csv')
city_data_df.head()

/Users/amitchoksi/Class/Weather_DatabaseWeather_database.csv


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,59.81,45,100,6.93,overcast clouds
1,1,Flinders,AU,-34.5833,150.8552,53.26,62,0,8.99,clear sky
2,2,Qaanaaq,GL,77.4840,-69.3632,30.24,72,100,2.37,overcast clouds
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,67.86,83,100,9.22,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.35,78,13,10.51,light rain


In [90]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [91]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,67.86,83,100,9.22,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.35,78,13,10.51,light rain
6,6,Kavieng,PG,-2.5744,150.7967,81.90,72,6,14.00,clear sky
7,7,Lata,PT,40.1629,-8.3327,77.81,55,0,4.88,clear sky
14,14,Kijang,KR,35.2442,129.2139,65.93,88,0,9.75,clear sky
19,19,Yellowknife,CA,62.4560,-114.3525,66.72,59,40,18.41,scattered clouds
21,21,Mahebourg,MU,-20.4081,57.7000,73.69,73,75,10.36,broken clouds
23,23,Atakpame,TG,7.5333,1.1333,72.95,93,93,6.06,overcast clouds
24,24,Saint-Philippe,RE,-21.3585,55.7679,67.59,79,49,12.53,light rain
25,25,Kapaa,US,22.0752,-159.3190,78.78,84,75,12.66,broken clouds


In [92]:
# 4a. Determine if there are any empty rows.

df_nan = preferred_cities_df[ preferred_cities_df["Country"].isna()]

# show the dataframe
print(df_nan)


    City_ID         City Country     Lat      Lng  Max Temp  Humidity  \
97       97  Henties Bay     NaN -22.116  14.2845     66.16        59   

    Cloudiness  Wind Speed Current Description  
97           0        5.03           clear sky  


In [93]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

preferred_cities_df.dropna(subset=['Country'], inplace=True)

preferred_cities_df

/var/folders/79/qwd7z0dn71ncnmdc1lbd61qw0000gn/T/ipykernel_32275/736248334.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preferred_cities_df.dropna(subset=['Country'], inplace=True)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,67.86,83,100,9.22,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.35,78,13,10.51,light rain
6,6,Kavieng,PG,-2.5744,150.7967,81.90,72,6,14.00,clear sky
7,7,Lata,PT,40.1629,-8.3327,77.81,55,0,4.88,clear sky
14,14,Kijang,KR,35.2442,129.2139,65.93,88,0,9.75,clear sky
...,...,...,...,...,...,...,...,...,...,...
696,696,Ferme-Neuve,CA,46.7001,-75.4493,72.27,37,27,8.16,scattered clouds
698,698,Saint Albans,GB,51.7500,-0.3333,69.15,48,39,18.41,scattered clouds
699,699,Sinnamary,GF,5.3833,-52.9500,82.49,72,13,9.28,few clouds
703,703,Paracuru,BR,-3.4100,-39.0306,83.10,72,18,9.13,few clouds


In [94]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Arraial Do Cabo,BR,67.86,overcast clouds,-22.9661,-42.0278,
4,Rikitea,PF,74.35,light rain,-23.1203,-134.9692,
6,Kavieng,PG,81.90,clear sky,-2.5744,150.7967,
7,Lata,PT,77.81,clear sky,40.1629,-8.3327,
14,Kijang,KR,65.93,clear sky,35.2442,129.2139,
19,Yellowknife,CA,66.72,scattered clouds,62.4560,-114.3525,
21,Mahebourg,MU,73.69,broken clouds,-20.4081,57.7000,
23,Atakpame,TG,72.95,overcast clouds,7.5333,1.1333,
24,Saint-Philippe,RE,67.59,light rain,-21.3585,55.7679,
25,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,


In [95]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")



Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [98]:
# 7. Drop the rows where there is no Hotel Name.

df_nan = hotel_df[ hotel_df["Hotel Name"].isna()]

hotel_df.dropna(subset=['Hotel Name'], inplace=True)

hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Arraial Do Cabo,BR,67.86,overcast clouds,-22.9661,-42.0278,Pousada Porto Praia
4,Rikitea,PF,74.35,light rain,-23.1203,-134.9692,People ThankYou
6,Kavieng,PG,81.90,clear sky,-2.5744,150.7967,Nusa Island Retreat
7,Lata,PT,77.81,clear sky,40.1629,-8.3327,Casa Maquia
14,Kijang,KR,65.93,clear sky,35.2442,129.2139,Samyang Beach Villa
...,...,...,...,...,...,...,...
696,Ferme-Neuve,CA,72.27,scattered clouds,46.7001,-75.4493,Pourvoirie Club Fontbrune Ltee
698,Saint Albans,GB,69.15,scattered clouds,51.7500,-0.3333,The White Hart
699,Sinnamary,GF,82.49,few clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
703,Paracuru,BR,83.10,few clouds,-3.4100,-39.0306,Hotel Vento Brasil


In [99]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [100]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

#city name
#country code
#weather description
# maximum temperature
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [101]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))